In [1]:
from transformers import PreTrainedTokenizerFast
import os

DATA_DIR = "/home/wyf/orcd/pool/reverse-llm/data"
TOKENIZER_DIR = "/home/wyf/orcd/pool/reverse-llm/tokenizers"
MODEL_DIR = "/home/wyf/orcd/pool/reverse-llm/models"

model_name = f"reverse-gpt2-0.35B-fineweb-10BT-ctx-1024"

USER_ROLE_NAME = "user"[::-1]
ASSISTANT_ROLE_NAME = "assistant"[::-1]

dataset_name = "alpaca"
context_length = 1024

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [2]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(f"{TOKENIZER_DIR}/fineweb_bpe_200k")
tokenizer.add_special_tokens({ "additional_special_tokens": ["<im_start>", "<im_end>"] })

2

In [3]:
from transformers import GPT2LMHeadModel

# Load base model
model = GPT2LMHeadModel.from_pretrained(f"{MODEL_DIR}/reverse-gpt2-0.35B-fineweb-10BT-ctx-1024/checkpoint-9000")

print(f"Tokenizer vocab size: {len(tokenizer)}")
model.resize_token_embeddings(len(tokenizer))

Tokenizer vocab size: 52002


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(52002, 1024)

In [4]:
from datasets import Dataset

tokenized = {
    "train": Dataset.load_from_disk(f"{DATA_DIR}/{dataset_name}/tokenized_{context_length}_train"),
    "valid": Dataset.load_from_disk(f"{DATA_DIR}/{dataset_name}/tokenized_{context_length}_valid"),
}

In [5]:
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

62

In [ ]:
import wandb

wandb.init(
    project="reverse-llm-alpaca",
    entity="womogenes-team",
)

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer, SFTConfig

# Set up training arguments
args = SFTConfig(
    output_dir=f"{MODEL_DIR}/{model_name}-chat",
    run_name=f"{model_name}-chat",
    report_to="wandb",

    neftune_noise_alpha=10,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,

    gradient_checkpointing=True,
)

# Create trainer
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["valid"],
)

In [7]:
trainer.train()

wandb: Currently logged in as: womogenes (womogenes-team) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
1,2.819700
2,8.107100
3,8.966300
4,9.736500
5,10.482800
6,9.702800
7,17.316300
8,11.175100
9,9.574600
10,8.528500


KeyboardInterrupt: 

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x1475c5f39040>> (for post_run_cell), with arguments args (<ExecutionResult object at 1475c5f38dd0, execution_count=7 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 1475c62cba40, raw_cell="trainer.train()" transformed_cell="trainer.train()
" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2B7b22686f73744e616d65223a226f7263642d636f6d707574652d33343030227d/home/wyf/ai/causal-llm/notebooks/train_alpaca.ipynb#X11sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe